In [2]:
# load all JSONL files in the directory
import os
import json

dir_path = "/mnt/fast10/brandon/mmr_rollout_data/final_combined_MC_and_verification_files_updated_rollouts"

first_jsonl = next((f for f in os.listdir(dir_path) if f.endswith(".jsonl")), None)
if first_jsonl:
    with open(os.path.join(dir_path, first_jsonl), "r") as f:
        first_line = f.readline()
        if first_line:
            columns = list(json.loads(first_line).keys())
        else:
            columns = []
else:
    columns = []

print(columns)

['prompt_uid', 'rollout_question', 'rollout_response', 'rollout_ground_truth_answer', 'rollout_steps_with_score', 'rollout_image_path', 'rollout_uuid', 'o4_mini_verification_custom_id', 'o4_mini_verification_solution', 'o4_mini_isVerified', 'gpt_4.1_mini_verification_custom_id', 'gpt_4.1_mini_verification_solution', 'gpt_4.1_mini_isVerified', 'gpt_4.1_nano_verification_custom_id', 'gpt_4.1_nano_verification_solution', 'gpt_4.1_nano_isVerified']


In [ ]:
for file in os.listdir(dir_path):
    if file.endswith(".jsonl"):
        with open(os.path.join(dir_path, file), "r") as f:
            for line in f:
                item = json.loads(line)